Re work [README example](https://github.com/WinVector/data_algebra) using Polars driver.

In [1]:
import polars as pl
import data_algebra


data_algebra.__version__

'1.5.0'

In [2]:
d_polars = pl.DataFrame({
    'subjectID':[1, 1, 2, 2],
    'surveyCategory': [ "withdrawal behavior", "positive re-framing", "withdrawal behavior", "positive re-framing"],
    'assessmentTotal': [5., 2., 3., 4.],
    'irrelevantCol1': ['irrel1']*4,
    'irrelevantCol2': ['irrel2']*4,
})

d_polars

subjectID,surveyCategory,assessmentTotal,irrelevantCol1,irrelevantCol2
i64,str,f64,str,str
1,"""withdrawal beh...",5.0,"""irrel1""","""irrel2"""
1,"""positive re-fr...",2.0,"""irrel1""","""irrel2"""
2,"""withdrawal beh...",3.0,"""irrel1""","""irrel2"""
2,"""positive re-fr...",4.0,"""irrel1""","""irrel2"""


In [3]:
scale = 0.237

ops = (
    data_algebra.descr(d=d_polars)
        .extend({'probability': f'(assessmentTotal * {scale}).exp()'})
        .extend({'total': 'probability.sum()'},
                partition_by='subjectID')
        .extend({'probability': 'probability / total'})
        .extend({'row_number': '(1).cumsum()'},
                partition_by=['subjectID'],
                order_by=['probability'], 
                reverse=['probability'])
        .select_rows('row_number == 1')
        .select_columns(['subjectID', 'surveyCategory', 'probability'])
        .map_columns({'surveyCategory': 'diagnosis'})
        .order_rows(["subjectID"])
    )

In [4]:
ops.transform(d_polars)

subjectID,diagnosis,probability
i64,str,f64
1,"""withdrawal beh...",0.670622
2,"""positive re-fr...",0.558974
